---
layout: code-post
title: AlphaZero for Brandub
description: We provide a from scratch implementation of AlphaZero to learn how to play brandub.
tags: [tafl, neural nets]
---

In this post I'm going to explore how to implement [AlphaZero](https://arxiv.org/pdf/1712.01815.pdf)
to learn how to play Brandub (a tafl variant). I previously put together a [Brandub package](https://github.com/kevinnowland/brandub)
which I won't use directly, but whose code I will modify as needed. You can find the rules of the
game there.

The main purpose of this is to prove that I understand the algorithm well enough to implement
a very crude version of it. How crude? I doubt the best trained player after even a few days of training
will be able to play very well. My big question is whether the trained model can beat me, who has
never played a full game before despite all this coding that I've done.


## Algorithm description

### Game play technology

AlphaZero plays games using two main pieces of technology. The first is a neural network $f\_\theta(s)$ with
parameters $\theta$ which takes the current game state $s$ and outputs $(p, z)$ where where $p$ is a probability
vector of all possible moves and $z \in [-1, 1]$ is an estimated game score for the current player with -1 being a
loss, 0 a draw, and 1 a win. Only the neural network parameters $\theta$ will be learned.

One could play 
just using $f\_\theta$ by choosing moves according to the output move probabilities $p$, 
but instead of doing this, we rely on
the second piece of technology, a Monte Carlo Tree Search (MCTS) algorithm. The tree search
uses the neural network to explore the space of moves and choose the best one taking into account
how the game might proceed. The neural network output, as we will
see later, is extremely raw and can suggest moves which are illegal (moving an opponent's piece) or 
even impossible (moving off the board). In addition to suggestiongm oves which will likely lead to victory
for the curernt player, the tree search algorithm helps encode the rules of the game
by by only exploring using legal moves. The tree
search is not learned directly, but does take as inputs the neural network $f\_\theta$ and current game state. 
The output is a policy $\pi$, a probability vector that is used to select the next move.

### Learning

Give the neural network and MCTS algorithm, play proceeds until a game ends at step $T$.
At each step $t$ of the game we have neural network output $(p\_t, v\_t)$ as well as 
the policy $\pi\_t$ governing move selection. The final game result is $s\_T \in \{-1, 0, 1\}$. 
The loss $\ell\_t$ for a step is
$$
  \ell\_t = (v\_t - s\_T)^2 - \pi\_t^t\log p\_t + c \|\theta\|^2,
$$
where $c$ is an $\ell^2$-regularization parameter. Thus we are looking at mean squared
error for the 
predicted output $v$ with cross-entropy forcing predicted moved probabilities $p$ to look like 
the MCTS policy $\pi$ with a regularization term.
The overall loss function is the average of this over the $T$ moves making up the game.

After each game, we backpropagate from the loss function through the nerual network to
change the neural network parameters $\theta$. Note that while policies $\pi\_t$ do depend on 
$\theta$ since the MCTS takes the nerual network as input, we will pretend it does not and thus 
does not affect backpropogation.


## Implementation

### Game State

The state of a game is encoded similarly but not identically to what was done in the brandub
package. The state tensor has is a stack of 7x7 binary tensors and some constant tensors.
The first layer encodes the current player's pawns and the second layer is the current player's
monarch. The current player might be the attacker and not have a monarch, but that is fine and
the input plane will be all zero. These layers repeat with the opponent's positions. 
AlphaZero maintains 8 timesteps of history, but we will try to keep only 3, if that. While the connections
from these planes into the neural net
will be learned and truly will be input, I suspect their main use will be checking to see if a 
position has repeated leading to a draw. AlphaZero for Chess contains two more constant planes, 
either 1 or 0, depending on if the current position has repeated once or twice before.
Probably the order matters, although I am not sure. Since brandub ends in a draw with only a 
single repeated position, we do not need to encode this information for the player.

In addition we will include a constant plane that is not repeated to indicate which player is
playing, either 0 for defense or 1 for attack.

Overall, this leads to a game state tensor of size $7\times7\times(4\times3 + 1)$, i.e.,
we have 637 features.

In [31]:
import torch

def get_initial_game_state():
    """returns pytorch tensor encoding initial brandub game state"""
    
    game_state = torch.zeros([7, 7, 13], dtype=torch.float64)
    
    # attacking pawns
    game_state[3, 0, 0] = 1
    game_state[3, 1, 0] = 1
    game_state[3, 5, 0] = 1
    game_state[3, 6, 0] = 1
    game_state[0, 3, 0] = 1
    game_state[1, 3, 0] = 1
    game_state[5, 3, 0] = 1
    game_state[6, 3, 0] = 1
    
    # defensive pawns
    game_state[3, 2, 2] = 1
    game_state[3, 4, 2] = 1
    game_state[2, 3, 2] = 1
    game_state[4, 3, 2] = 1
    
    # defensive monarch
    game_state[3, 3, 3] = 1
    
    # attack's turn
    game_state[:, :, 12] = 1
    
    return game_state

Now we write some functions to check for end game conditions. While I won't be optimizing this
code too much, I won't bother to check types and board legality.

In [299]:
def get_monarch_index(game_state):
    """ return which game state layer encodes the defensive monarch 
    for the current board """
    if game_state[0, 0, 12] == 1:
        return 3
    else:
        return 1

    
def attack_victory(board, monarch_index):
    """ attack wins if the monarch is not on the current board"""
    
    return board[:, :, monarch_index].max() == 0


def defense_victory(board, monarch_index):
    """ defense wins if the monarch is in the forest """
    
    return board[0, 0, monarch_index] == 1 or \
        board[0, 6, monarch_index] == 1 or \
        board[6, 0, monarch_index] == 1 or \
        board[6, 6, monarch_index] == 1


def board_shadow(board, is_attack_turn):
    """ return a 7x7 tensor encoding current state with
    -1 being an attacking pawn, 1 a defensive pawn,
    2 the monarch.
    
    board is a 7x7x4 tensor
    is_attack_turn is a boolean
    
    This is used to look for draws with game states"""
    
    raw_shadow = board[:, :, 0] + 2 * board[:, :, 1] - \
        board[:, :, 2] - 2 * board[:, :, 3]
    
    if is_attack_turn:
        return -1 * raw_shadow
    else:
        return raw_shadow
    
    
def game_shadow(game_state):
    """ returns the board shadow for the current board
    and player. """
    
    return board_shadow(game_state[:, :, :4], game_state[0, 0, 12] == 1)
        

def drawn_game(board, game_state):
    """ take a new board and compare to the boards in the game state
    to see if it would result in a draw.
    
    NOTE: This assumes that the current player for board is THE SAME AS
    the current player in the game_state. 
    
    NOTE: Do not bother checking most recent game state, as we
    assume the presented board is after a move. """
    
    is_attack_turn = game_state[0, 0, 12] == 1
    new_board_shadow = board_shadow(board, is_attack_turn)
    
    # check two moves ago
    board_shadow_1 = board_shadow(game_state[:, :, 4:8], is_attack_turn)
    
    if torch.all(torch.eq(new_board_shadow, board_shadow_1)):
        return True
    else:
        board_shadow_2 = board_shadow(game_state[:, :, 8:12],
                                     not is_attack_turn)
        return torch.all(torch.eq(new_board_shadow, board_shadow_2))

In [300]:
initial_game_state = get_initial_game_state()
print(game_shadow(initial_game_state))

tensor([[-0., -0., -0., -1., -0., -0., -0.],
        [-0., -0., -0., -1., -0., -0., -0.],
        [-0., -0., -0.,  1., -0., -0., -0.],
        [-1., -1.,  1.,  2.,  1., -1., -1.],
        [-0., -0., -0.,  1., -0., -0., -0.],
        [-0., -0., -0., -1., -0., -0., -0.],
        [-0., -0., -0., -1., -0., -0., -0.]], dtype=torch.float64)


### Movement

Movement will be encoded as a $7\times7\times24$ tensor. The first two coordinates
indicate the position on the board where a piece should be found and the
final coordinate indicates the direction. The value modulo 6 then adding 1 (not
modulo 6) is the amount of spaces to move, raw values between 0 and 5 indicate
moving down, 6 to 11 moving up, 12 to 17 moving right, 18 to 23 moving
left. $(7, 7, 4)$ will also be the shape of the probability vector put out by the neural net and
will include invalid moves. The first step of the MCTS will be to reduce this
only to valid moves. A singular move will be an index tensor that we can samples
using the `Categorical` object in `torch.distributions.categorical`.

We will need auxiliary functions to find valid moves and then perform
those moves. Again, since we're trying to remove unnecessary cruft, we will
not verify many things, such as piece existence. In the following code block
we will write auxiliary functions involved with finding valid moves
and the new position of a piece given some move.

In [370]:
def is_forest(position_2d):
    """ returns whether the first two coords of
    position_2d is a forest"""
    return position_2d[0] in [0, 6] and position_2d[1] in [0, 6]


def is_castle(position_2d):
    """ returns whether the first two coords of position_2d
    is a castle"""
    return position_2d[0] == 3 and position_2d[1] == 3
   

def find_piece_plane(board, position_2d, planes=[2, 3, 0, 1]):
    """ returns the plane of piece at given (i, j) position """
    for k in planes:
        if board[position_2d[0], position_2d[1], k] == 1:
            return k
        
        
def find_piece_at_position(board, position_2d):
    """ returns the (i, j, k) position of the piece at position (i, j) 
    if some is present else returns None """
    
    k = find_piece_plane(board, position_2d, [2, 3, 0, 1])
    if k is not None:
        return (position_2d[0], position_2d[1], k)
    else:
        return None

        
def valid_moves(board, position):
    """ find valid movement indices 0-24 for a piece
    that is (assumed to be) at position, which is a tensor with length 2"""
    
    is_pawn = board[position[0], position[1], 1] == 0
    
    shadow = board[:, :, 0] + board[:, :, 1] + board[:, :, 2] + board[:, :, 3]
    
    def check_direction(direction_vector):
        """ get valid moves in the given direction
        direction must be [+/-1, 0] or [0, +/-1] torch tensors
        """

        valid_moves = []

        coord = 0 if direction_vector[0] != 0 else 1
        positive_direction = direction_vector[coord] == 1
        end_value = 6 if positive_direction else 0
        
        if coord == 0:
            if positive_direction:
                base = 0
            else:
                base = 6
        else:
            if positive_direction:
                base = 12
            else:
                base = 18

        keep_going = position[coord] != end_value
        i = 0
        while keep_going:
            i += 1

            new_pos = position + i * direction_vector

            # stop if run into a piece
            if shadow[tuple(new_pos)] == 1:
                break

            # ignore the castle
            if is_castle(new_pos):
                continue

            keep_going = new_pos[coord] != end_value

            # if pawn and at the wall, see if its a forest but don't add
            if not keep_going and is_pawn and is_forest(new_pos):
                break

            valid_moves.append(base + i - 1)

        return valid_moves
    
    direction_vectors = (
        torch.tensor([1, 0]),
        torch.tensor([-1, 0]),
        torch.tensor([0, 1]),
        torch.tensor([0, -1])
    )

    return [
        direction
        for dvec in direction_vectors
        for direction in check_direction(dvec)
    ]
    
    
def all_valid_moves(board):
    """ get all indices  of valid moves. Only pieces on the first
    two planes can move. 
    
    Returns 7x7x24 binary tensor """
    
    # get all pieces that can move
    pos_moves = [
        (pos[0], pos[1], move_index)
        for pos in board[:, :, :2].nonzero()
        for move_index in valid_moves(board, pos[:2])
    ]
    
    moves = torch.zeros(7, 7, 24)
    for pos_move in pos_moves:
        moves[pos_move] = 1
        
    return moves


def find_new_position(board, move):
    """ find the piece at the given
    position and return a tuple containing
    the pieces new location.
    
    NOTE: assumes piece is on the first or second plane """
    
    # whether to move vertically or horizontally
    move_vertical = move[2] < 12
    
    # how much to move
    direction = 1 if move[2] % 12 < 6 else -1
    move_val = direction * (move[2] % 6 + 1)
    
    # find where the piece is
    plane = find_piece_plane(board, move[:2], [0, 1])
    
    if move_vertical:
        return (move[0] + move_val, move[1], plane)
    else:
        return (move[0], move[1] + move_val, plane)

Now we write functions that allow us to advance game state by performing a move
and then capturing the pieces.

In [371]:
def is_piece_captured(board, position):
    """ determine if a piece at (i, j, k) is captured.
    
    NOTE: does not check that a piece is at the position
    
    NOTE: This does not validate that the raw_board is a
    vaid raw_board. """
    
    i = position[0]
    j = position[1]

    topography_inds_corners = [(0, 0), (0, 6), (6, 0), (6, 6)]

    if position[2] == 0 or position[2] == 2:
        # pawn logic
        
        if position[2] == 0:
            enemy_board = board[:, :, 2] + board[:, :, 3]
        else:
            enemy_board = board[:, :, 0] + board[:, :, 1]
        
        if board[3, 3, 1] + board[3, 3, 3] == 0:
            # no monarch? then castle is threat
            topography_inds = topography_inds_corners + [(3, 3)]
        else:
            topography_inds = topography_inds_corners

        topography = torch.zeros(7, 7)
        for ind in topography_inds:
            topography[ind] = 1
        bad_things = enemy_board + topography

        if i == 0 or i == 6:
            # on top or bottom (can't be in corner)
            if bad_things[i, j-1] == 1 and bad_things[i, j+1] == 1:
                return True
            else:
                return False
        elif j == 0 or j == 6:
            # on left or right side (can't be in corner)
            if bad_things[i-1, j] == 1 and bad_things[i+1, j] == 1:
                return True
            else:
                return False
        else:
            # otherwise just check
            if bad_things[i, j-1] == 1 and bad_things[i, j+1] == 1:
                return True
            elif bad_things[i+1, j] == 1 and bad_things[i-1, j] == 1:
                return True
            else:
                return False

    elif position[2] == 1 or position[2] == 3:
        # monarch logic
        
        if position[2] == 1:
            enemy_board = board[:, :, 2]
        else:
            enemy_board = board[:, :, 0]

        if i != 3 or j != 3:
            # empty castles are threats
            topography_inds = topography_inds_corners + [(3, 3)]
        else:
            topography_inds = topography_inds_corners

        topography = torch.zeros(7, 7)
        for ind in topography_inds:
            topography[ind] = 1
        bad_things = enemy_board + topography

        if (i, j) in [(3, 2), (3, 3), (3, 4), (2, 3), (4, 3)]:
            # in or next to castle have to be surrounded
            if bad_things[i+1, j] == 1 and bad_things[i-1, j] == 1 \
                and bad_things[i, j+1] == 1 and bad_things[i, j-1] == 1:
                return True
            else:
                return False
        else:
            if (i, j) in [(0, 0), (0, 6), (6, 0), (6, 6)]:
                # safe in corner
                return False
            elif i == 0 or i == 6:
                # top or bottom: non corner
                if bad_things[i, j+1] == 1 and bad_things[i, j-1] == 1:
                    return True
                else:
                    return False
            elif j == 0 or j == 6:
                # left or right: non corner
                if bad_things[i+1, j] == 1 and bad_things[i-1, j] == 1:
                    return True
                else:
                    return False
            else:
                # any other spot on board
                if bad_things[i+1, j] == 1 and bad_things[i-1, j] == 1:
                    return True
                elif bad_things[i, j+1] == 1 and bad_things[i, j-1] == 1:
                    return True
                else:
                    return False
    else:
        msg = "position[2] must be in {0, 1, 2, 3}"
        raise Exception(msg)


def game_state_move(game_state, move):
    """ advance game state by moving piece, performing captures, 
    flipping players, and returning a new game state. Also
    returns if the game is over as needed.
    
    NOTE: asumes the move is valid but does not check 
    
    This takes about 0.55 ms"""
    
    attacker_is_playing = game_state[0, 0, 12] == 1
    
    # perform the move
    old_board = game_state[:, :, :4]
    new_pos = find_new_position(game_state[:, :, :4], move)
    old_pos = (move[0], move[1], new_pos[2])
    
    new_board_ = torch.zeros(7, 7, 4)
    new_board_[:, :, :] = old_board
    new_board_[old_pos] = 0
    new_board_[new_pos] = 1
    
    # check any pieces near the given piece to see if they are captured
    # if so, remove them from the new_board
    i = new_pos[0]
    j = new_pos[1]
    
    for check_pos in [(i, j), (i-1, j), (i+1, j), (i, j-1), (i, j+1)]:
        
        if check_pos[0] >= 0 and check_pos[0] <= 6 \
            and check_pos[1] >=0 and check_pos[1] <= 6:
            
            piece_pos_ = find_piece_at_position(new_board_, check_pos)
            
            if piece_pos_ is not None:
                if is_piece_captured(new_board_, piece_pos_):
                    new_board_[piece_pos_] = 0
                    
    
    # see if the new_board_ is in a victory state
    # return if so
    monarch_index = 3 if attacker_is_playing else 1
    
    if attack_victory(new_board_, monarch_index):
        end_game_state = 1 if attacker_is_playing else -1
    elif defense_victory(new_board_, monarch_index):
        end_game_state = -1 if attacker_is_playing else 1
    elif drawn_game(new_board_, game_state):
        end_game_state = 0
    else:
        end_game_state = None
        
    if end_game_state is not None:
        return None, end_game_state
    else:
        # flip players
        new_board = torch.zeros(7, 7, 4)
        new_board[:, :, :2] = new_board_[:, :, 2:]
        new_board[:, :, 2:] = new_board_[:, :, :2]
    
        # form the new game_state
        new_game_state = torch.zeros(7, 7, 13, dtype=torch.float64)

        # update boards
        new_game_state[:, :, :4] = new_board
        new_game_state[:, :, 4:8] = game_state[:, :, :4]
        new_game_state[:, :, 8:12] = game_state[:, :, 4:8]

        # update player
        new_game_state[:, :, 12] = 1 - game_state[:, :, 12]
    
    
        return new_game_state, None

We could now play a game starting from the initial game state provided by `get_initial_game_state()`
and applying moves using `game_state_move()`. We would have to be careful to only supply
valid moves though, because the board could easily get into an invalid state since we are
not verifying many things.